In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [8, 8]
import os
from skimage import feature
from skimage import exposure

from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle

In [ ]:
# for root, dirs, files in os.walk("images"):
#     path = root.split(os.sep)
# #     print((len(path) - 1) * '---', os.path.basename(root))
#     for file in files:
# #         print(len(path) * '---', file)
# #         image = cv2.imread(os.path.join())
#         image = cv2.imread(os.path.join(root, file))
#         image = cv2.resize(image, (200, 200))
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         cv2.imwrite(os.path.join(root, file), image)

# 0. Przykłady działania HOG

In [ ]:
def hog_descriptor(image, orientations, pixels_per_cell, cells_per_block):
    return feature.hog(image, orientations, pixels_per_cell, cells_per_block, transform_sqrt=True, block_norm="L1", visualize=True)

orientations=9
pixels_per_cell=(2, 2)
cells_per_block=(3, 3)

image = cv2.imread('images/bmw/1.png')

(H, hogImage) = hog_descriptor(image, orientations, pixels_per_cell, cells_per_block)

hogImage = exposure.rescale_intensity(hogImage, out_range=(0, 255))
hogImage = hogImage.astype("uint8")
plt.imshow(hogImage, cmap='gray')

In [ ]:
image = cv2.imread('images/ford/1.png')

(H, hogImage) = hog_descriptor(image, orientations, pixels_per_cell, cells_per_block)


hogImage = exposure.rescale_intensity(hogImage, out_range=(0, 255))
hogImage = hogImage.astype("uint8")
print(len(H))
plt.imshow(hogImage, cmap='gray')

In [ ]:
image = cv2.imread('images/ferrari/1.jpg')

(H, hogImage) = hog_descriptor(image, orientations, pixels_per_cell, cells_per_block)


hogImage = exposure.rescale_intensity(hogImage, out_range=(0, 255))
hogImage = hogImage.astype("uint8")
plt.imshow(hogImage, cmap='gray')

# 1. Wczytaj zbiór danych

In [ ]:
mapping = {}
images = []
labels = []

for i, brand in enumerate(os.listdir('images')):
    mapping[i] = brand
    brand_directory = os.path.join('images', brand)
    for filename in os.listdir(brand_directory):
        image = cv2.imread(os.path.join(brand_directory, filename))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images.append(image)
        labels.append(i)

print(labels)
print(mapping)

# 2. Wykonaj ekstrakcję cech za pomocą HOG

In [ ]:
orientations=9
pixels_per_cell=(3, 3)
cells_per_block=(3, 3)

images, labels = shuffle(images, labels)
features = [hog_descriptor(image, orientations, pixels_per_cell, cells_per_block)[0] for image in images]

In [ ]:
len(features[1])

# 3. Trenuj klasyfikator KNN

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(features, labels)

# 4. Testowanie skuteczności

In [ ]:
for filename in os.listdir('test_images'):
    image = cv2.imread(os.path.join('test_images', filename))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (300, 300))
    rescaled_image = cv2.resize(image, (200, 200))
    
    gray_image = cv2.cvtColor(rescaled_image, cv2.COLOR_BGR2GRAY)
    H = hog_descriptor(gray_image, orientations, pixels_per_cell, cells_per_block)[0]
    result = model.predict(H.reshape(1, -1))[0]
    
    cv2.putText(image, mapping[result], (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    plt.imshow(image)
    plt.show()